In [37]:
%pip install category_encoders

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [38]:
%pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [39]:
#Import all required libraries for data processing, ML modeling, and evaluation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, roc_auc_score

import category_encoders as ce
import xgboost as xgb

In [40]:
#️Load all OLIST e-commerce datasets into pandas DataFrames
items = pd.read_csv("olist_order_items_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")

print(items.shape, products.shape, orders.shape, payments.shape)

(112650, 7) (32951, 9) (99441, 8) (103886, 5)


In [41]:
#️Combine multiple payment rows into a single payment summary per order
payments_agg = payments.groupby("order_id").agg({
    "payment_value": "sum",
    "payment_installments": "mean"
}).reset_index()

payments_agg.columns = ["order_id", "order_payment_value", "avg_installments"]

In [42]:
#️Merge item, product, order, and payment data into one master dataset
df = items.merge(products, on="product_id", how="left")
df = df.merge(orders[["order_id", "order_purchase_timestamp"]], on="order_id", how="left")
df = df.merge(payments_agg, on="order_id", how="left")

In [59]:
df.shape

(112647, 25)

In [57]:
df.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,order_purchase_timestamp,order_payment_value,avg_installments,purchase_month,purchase_dayofweek,purchase_hour,volume_cm3,price_per_g,freight_ratio,is_holiday_season
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,19-09-2017 09:45,58.90,13.29,cool_stuff,58.0,598.0,...,2017-09-13 08:59:00,72.19,2.0,9,2,8,3528.0,0.090476,0.221870,0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,03-05-2017 11:05,239.90,19.93,pet_shop,56.0,239.0,...,2017-04-26 10:53:00,259.83,3.0,4,2,10,60000.0,0.007996,0.082731,0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,18-01-2018 14:48,199.00,17.87,moveis_decoracao,59.0,695.0,...,2018-01-14 14:33:00,216.87,5.0,1,6,14,14157.0,0.065225,0.089350,0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,15-08-2018 10:10,12.99,12.79,perfumaria,42.0,480.0,...,2018-08-08 10:00:00,25.78,2.0,8,2,10,2400.0,0.064627,0.914224,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,13-02-2017 13:57,199.90,18.14,ferramentas_jardim,59.0,409.0,...,2017-02-04 13:57:00,218.04,3.0,2,5,13,42000.0,0.053292,0.090294,0


In [44]:
#️Convert timestamps and extract time-based features for seasonality analysis
#️Convert timestamp column to datetime FIRST
df["order_purchase_timestamp"] = pd.to_datetime(
    df["order_purchase_timestamp"],
    format="%d-%m-%Y %H:%M",
    errors="coerce"
)

#️Now safely extract time-based features
df["purchase_month"] = df["order_purchase_timestamp"].dt.month
df["purchase_dayofweek"] = df["order_purchase_timestamp"].dt.dayofweek
df["purchase_hour"] = df["order_purchase_timestamp"].dt.hour

#️Drop rows where critical values are missing
df = df.dropna(subset=["price", "order_payment_value"])

In [45]:
#️Create business-relevant features like product volume, cost density, and holiday flags
df["volume_cm3"] = (
    df["product_length_cm"] *
    df["product_height_cm"] *
    df["product_width_cm"]
)

df["price_per_g"] = df["price"] / (df["product_weight_g"] + 1)
df["freight_ratio"] = df["freight_value"] / (df["price"] + 1)
df["is_holiday_season"] = df["purchase_month"].isin([11, 12]).astype(int)

In [61]:
df

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,...,order_purchase_timestamp,order_payment_value,avg_installments,purchase_month,purchase_dayofweek,purchase_hour,volume_cm3,price_per_g,freight_ratio,is_holiday_season
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,19-09-2017 09:45,58.90,13.29,cool_stuff,58.0,598.0,...,2017-09-13 08:59:00,72.19,2.0,9,2,8,3528.0,0.090476,0.221870,0
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,03-05-2017 11:05,239.90,19.93,pet_shop,56.0,239.0,...,2017-04-26 10:53:00,259.83,3.0,4,2,10,60000.0,0.007996,0.082731,0
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,18-01-2018 14:48,199.00,17.87,moveis_decoracao,59.0,695.0,...,2018-01-14 14:33:00,216.87,5.0,1,6,14,14157.0,0.065225,0.089350,0
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,15-08-2018 10:10,12.99,12.79,perfumaria,42.0,480.0,...,2018-08-08 10:00:00,25.78,2.0,8,2,10,2400.0,0.064627,0.914224,0
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,13-02-2017 13:57,199.90,18.14,ferramentas_jardim,59.0,409.0,...,2017-02-04 13:57:00,218.04,3.0,2,5,13,42000.0,0.053292,0.090294,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,02-05-2018 04:11,299.99,43.41,utilidades_domesticas,43.0,1002.0,...,2018-04-23 13:57:00,343.40,1.0,4,0,13,53400.0,0.029553,0.144224,0
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,20-07-2018 04:31,350.00,36.53,informatica_acessorios,31.0,232.0,...,2018-07-14 10:26:00,386.53,1.0,7,5,10,44460.0,0.039102,0.104074,0
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,30-10-2017 17:14,99.90,16.95,esporte_lazer,43.0,869.0,...,2017-10-23 17:07:00,116.85,3.0,10,0,17,9576.0,0.103202,0.167988,0
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,21-08-2017 00:04,55.99,8.72,informatica_acessorios,56.0,1306.0,...,2017-08-14 23:02:00,64.71,3.0,8,0,23,8000.0,0.554356,0.153009,0


In [46]:
#️Define features and regression target
features = [
    "freight_value",
    "avg_installments",
    "purchase_month",
    "purchase_dayofweek",
    "purchase_hour",
    "product_weight_g",
    "volume_cm3",
    "price_per_g",
    "freight_ratio",
    "is_holiday_season",
    "product_category_name"
]

X = df[features]
y = df["price"]

In [60]:
X

,freight_value,avg_installments,purchase_month,purchase_dayofweek,purchase_hour,product_weight_g,volume_cm3,price_per_g,freight_ratio,is_holiday_season,product_category_name
0,13.29,2.0,9,2,8,650.0,3528.0,0.090476,0.221870,0,cool_stuff
1,19.93,3.0,4,2,10,30000.0,60000.0,0.007996,0.082731,0,pet_shop
2,17.87,5.0,1,6,14,3050.0,14157.0,0.065225,0.089350,0,moveis_decoracao
3,12.79,2.0,8,2,10,200.0,2400.0,0.064627,0.914224,0,perfumaria
4,18.14,3.0,2,5,13,3750.0,42000.0,0.053292,0.090294,0,ferramentas_jardim
...,...,...,...,...,...,...,...,...,...,...,...
112645,43.41,1.0,4,0,13,10150.0,53400.0,0.029553,0.144224,0,utilidades_domesticas
112646,36.53,1.0,7,5,10,8950.0,44460.0,0.039102,0.104074,0,informatica_acessorios
112647,16.95,3.0,10,0,17,967.0,9576.0,0.103202,0.167988,0,esporte_lazer
112648,8.72,3.0,8,0,23,100.0,8000.0,0.554356,0.153009,0,informatica_acessorios


In [47]:
#️Split data for regression
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [48]:
#️Define preprocessing steps to handle missing values, scaling, and categorical encoding
numeric_features = X_train.select_dtypes(include=np.number).columns
categorical_features = ["product_category_name"]

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ("target", ce.TargetEncoder())
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

In [49]:
#️Apply preprocessing to convert raw data into ML-ready numerical features
X_train_p = preprocessor.fit_transform(X_train, y_train)
X_test_p = preprocessor.transform(X_test)

In [50]:
#️Train an XGBoost regression model to learn optimal product pricing patterns
price_model = xgb.XGBRegressor(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42
)

price_model.fit(X_train_p, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

In [51]:
#️Evaluate the pricing model using RMSE and R² metrics
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

price_preds = price_model.predict(X_test_p)

rmse = np.sqrt(mean_squared_error(y_test, price_preds))
r2 = r2_score(y_test, price_preds)

print("RMSE:", rmse)
print("R²:", r2)


RMSE: 23.18393078444817
R²: 0.9845195154456721


In [69]:
#️Negotiation price recommender using price elasticity heuristic
def recommend_price_option3(sample_df, listed_price):
    sample_p = preprocessor.transform(sample_df)

    expected_price = price_model.predict(sample_p)[0]

    # Heuristic acceptance probability (price elasticity assumption)
    deviation = abs(listed_price - expected_price) / expected_price
    acceptance_score = max(0, 1 - deviation)

    # Negotiation logic
    min_price = expected_price * 0.9
    recommended_price = expected_price * (0.9 + 0.1 * acceptance_score)

    return {
        "listed_price": round(listed_price, 2),
        "expected_price": round(expected_price, 2),
        "heuristic_acceptance_score": round(acceptance_score, 2),
        "recommended_negotiation_price": round(recommended_price, 2)
    }

In [70]:
#️Percentage of predictions within ±10% of true price
tolerance = 0.10
within_range = np.abs(price_preds - y_test) <= tolerance * y_test

pseudo_accuracy = within_range.mean()
print("Predictions within ±10% of true price:", pseudo_accuracy)

Predictions within ±10% of true price: 0.967953839325344


In [71]:
#price_preds

In [72]:
for i in [5, 25, 100, 250]:
    sample = X_test.iloc[[i]]
    actual_price = y_test.iloc[i]

    result = recommend_price_option3(sample, actual_price)

    print(f"\n🔹 Sample Index: {i}")
    print(f"   Actual Listed Price        : ₹{actual_price:.2f}")
    print(f"   Expected Fair Price (ML)   : ₹{result['expected_price']:.2f}")
    print(f"   Buyer Acceptance Score     : {result['heuristic_acceptance_score']:.2f}")
    print(f"   Recommended Negotiation ₹  : ₹{result['recommended_negotiation_price']:.2f}")
    print("-" * 60)


🔹 Sample Index: 5
   Actual Listed Price        : ₹144.90
   Expected Fair Price (ML)   : ₹146.20
   Buyer Acceptance Score     : 0.99
   Recommended Negotiation ₹  : ₹146.07
------------------------------------------------------------

🔹 Sample Index: 25
   Actual Listed Price        : ₹649.90
   Expected Fair Price (ML)   : ₹659.19
   Buyer Acceptance Score     : 0.99
   Recommended Negotiation ₹  : ₹658.26
------------------------------------------------------------

🔹 Sample Index: 100
   Actual Listed Price        : ₹401.70
   Expected Fair Price (ML)   : ₹403.90
   Buyer Acceptance Score     : 0.99
   Recommended Negotiation ₹  : ₹403.68
------------------------------------------------------------

🔹 Sample Index: 250
   Actual Listed Price        : ₹112.99
   Expected Fair Price (ML)   : ₹108.19
   Buyer Acceptance Score     : 0.96
   Recommended Negotiation ₹  : ₹107.71
------------------------------------------------------------
